In [ ]:
import yaml

def load_dialog(patient_id):
    PASS = "first"
    with open(
        f"Dataset-Hucam-Nefro/patient_{patient_id:03d}/patient_{patient_id:03d}_transcription_{PASS}_pass.txt",
        "r",
    ) as f:
        prediction = f.read()
    return prediction


def format_dict(data, indent=0):
    """
    Recursively formats a dictionary or list into a readable string.
    """
    formatted = []
    prefix = "  " * indent  # Indentation for nesting
    if isinstance(data, dict):
        for key, value in data.items():
            formatted.append(f"{prefix}{key}:")
            formatted.append(format_dict(value, indent + 1))
    elif isinstance(data, list):
        for item in data:
            formatted.append(f"{prefix}- {format_dict(item, indent + 1).strip()}")
    else:
        # For strings, numbers, or other types
        formatted.append(f"{prefix}{data}")
    return "\n".join(formatted)


def readable_string(data):
    if isinstance(data, dict):
        # return json.dumps(data, indent=2, ensure_ascii=False) 
        return format_dict(data)
    return data

def get_diagnosis(hd):
    try:
        stage = hd["estagio_drc"]
        drc = ""
        drc += f"{stage['grau']}"
        if et := hd.get("etiologia_doença_de_base"):
            drc += " " + et
        return drc
    except TypeError:
        return hd


def get_field(summary, field):
    field = summary["relatorio_consulta_ambulatorial_nefrologia"][field]
    if isinstance(field, list):
        field = [readable_string(x) for x in field]
        return "\n".join(field)
    else:
        return field


def get_summary(patient_id):
    with open(
        f"Dataset-Hucam-Nefro/patient_{patient_id:03d}/patient_{patient_id:03d}_medical_summary.yaml"
    ) as fp:
        return yaml.safe_load(fp)
    
def get_prediction(patient_id):
    with open(
        f"results/patient_{patient_id}/outputs.yaml"
    ) as fp:
        return yaml.safe_load(fp)

def load_yaml(file_path):
    """Load a YAML file and return its content."""
    with open(file_path, "r", encoding="utf-8") as f:
        return yaml.safe_load(f)


In [ ]:
# import pandas as pd

# # Define prompt template for the medical evaluator
# prompt_template = (
#     "You are a medical evaluator tasked with comparing two texts related to medical diagnoses, clinical impressions, or proposed conduct. "
#     "Your job is to assign a similarity score from 0 to 10, where 0 indicates complete discrepancy (e.g., antonyms or unrelated terms) "
#     "and 10 indicates total similarity (e.g., synonymous terms or identical meanings). "
#     "Consider the context, medical terminology, and overall meaning.\n\n"
#     "Ground Truth: {ground_truth}\n"
#     "Prediction: {prediction}\n\n"
#     "Score: "
# )

# def evaluate_similarity_with_prompt(ground_truth, prediction):
#     """Evaluate similarity between ground truth and prediction using the model."""
#     prompt = prompt_template.format(ground_truth=ground_truth, prediction=prediction)
#     inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to(model.device)

#     # Generate response
#     with torch.no_grad():
#         outputs = model.generate(**inputs, max_length=256)
#     response = tokenizer.decode(outputs[0], skip_special_tokens=True)

#     # Extract the score from the response
#     try:
#         score = float(response.split("Score:")[-1].strip().split()[0])
#     except (ValueError, IndexError):
#         score = None

#     return score

# def evaluate_patient(patient_id):
#     """Evaluate a single patient's predictions against the ground truth."""
#     # Load YAML files
#     ground_truth = get_summary(patient_id)
#     prediction = get_prediction(patient_id)

#     # Extract fields
#     gt_diagnosis = get_diagnosis(ground_truth["relatorio_consulta_ambulatorial_nefrologia"]["hipoteses_diagnosticas"])
#     pred_diagnosis = get_diagnosis(prediction["hipoteses_diagnosticas"])

#     # gt_conduct = "\n".join(readable_string(x) for x in get_field(ground_truth["relatorio_consulta_ambulatorial_nefrologia"], "conduta"))
#     # pred_conduct = "\n".join(readable_string(x) for x in get_field(prediction, "conduta"))

#     # gt_impression = "\n".join(readable_string(x) for x in ground_truth["relatorio_consulta_ambulatorial_nefrologia"].get("impressão", []))
#     # pred_impression = "\n".join(readable_string(x) for x in prediction.get("impressão", []))

#     # Evaluate similarity
#     diagnosis_score = evaluate_similarity_with_prompt(gt_diagnosis, pred_diagnosis)
#     # conduct_score = evaluate_similarity_with_prompt(gt_conduct, pred_conduct)
#     # impression_score = evaluate_similarity_with_prompt(gt_impression, pred_impression)

#     # Compile results
#     return {
#         "patient_id": patient_id,
#         "GT Diagnosis": gt_diagnosis,
#         "Pred Diagnosis": pred_diagnosis,
#         "Diagnosis Similarity": diagnosis_score,
#         # "GT Conduct": gt_conduct,
#         # "Pred Conduct": pred_conduct,
#         # "Conduct Similarity": conduct_score,
#         # "GT Impression": gt_impression,
#         # "Pred Impression": pred_impression,
#         # "Impression Similarity": impression_score
#     }

# # Iterate over patients and compile results
# results = []
# for patient_id in range(1, 17):
#     print(f"Evaluating patient {patient_id}")
#     patient_results = evaluate_patient(patient_id)
#     if patient_results:
#         results.append(patient_results)

# # Save results to a DataFrame
# results_df = pd.DataFrame(results)
# results_df.to_csv("medical_similarity_results.csv", index=False)
# print("Evaluation complete. Results saved to 'medical_similarity_results.csv'.")


In [ ]:
import transformers
import torch


cache_dir = "/mnt/external8tb/models"
model_id = "aaditya/OpenBioLLM-Llama3-70B"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
    cache_dir=cache_dir,
    local_files_only=True,  # Force local loading
)

In [ ]:
import transformers
import torch

# Path to the resolved snapshot directory
model_dir = "/mnt/external8tb/models/models--aaditya--OpenBioLLM-Llama3-70B/snapshots/7ad17ef0d2185811f731f89d20885b2f99b1e994"

# Load the model and tokenizer explicitly
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_dir,
    torch_dtype=torch.bfloat16,
    local_files_only=True,
    device_map="auto", 
)

tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_dir,
    local_files_only=True,
)

# Create the pipeline
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    # device_map="auto",
)

# Test the pipeline
output = pipeline("Hello, world!")
print(output)


In [ ]:
# Define your own chat prompt
def create_chat_prompt(messages):
    """
    Concatenate messages in a chat format for text generation.
    """
    prompt = ""
    for message in messages:
        role = message["role"]
        content = message["content"]
        if role == "system":
            prompt += f"System: {content}\n"
        elif role == "user":
            prompt += f"User: {content}\n"
        elif role == "assistant":
            prompt += f"Assistant: {content}\n"
    prompt += "Assistant: "  # Add the generation prompt for the assistant
    return prompt


# for patient_id in range(1, 17):
for patient_id in [17]:

    with open(f"results/patient_{patient_id}/condensed_dialog.txt", "r", encoding="utf-8") as f:
        condensed_dialog = f.read()

    summary = get_summary(patient_id)

    # Get fields from the summary
    fields = {
        "historico_patologico_pregresso": get_field(summary, "historico_patologico_pregresso"),
        "exames_laboratoriais": get_field(summary, "exames_laboratoriais"),
        "exames_complementares": get_field(summary, "exames_complementares"),
        "exame_fisico": get_field(summary, "exame_fisico"),
        "medicamentos_em_uso": get_field(summary, "medicamentos_em_uso"),
        "identificacao_paciente": get_field(summary, "identificacao_paciente"),
    }

    concatenated_info = (
        f"**Identificação do Paciente**\n{fields['identificacao_paciente']}\n\n"
        f"**Histórico Patológico Pregresso**\n{fields['historico_patologico_pregresso']}\n\n"
        f"**Exames Laboratoriais**\n{fields['exames_laboratoriais']}\n\n"
        # f"**Exames Complementares**\n{fields['exames_complementares']}\n\n"
        # f"**Exame Físico**\n{fields['exame_fisico']}\n\n"
        # f"**Medicamentos em Uso**\n{fields['medicamentos_em_uso']}\n\n"
    )

    ground_truth = get_summary(patient_id)
    prediction = get_prediction(patient_id)


    # Extract fields
    # GT = get_diagnosis(ground_truth["relatorio_consulta_ambulatorial_nefrologia"]["hipoteses_diagnosticas"])
    # PRED = get_diagnosis(prediction["hipoteses_diagnosticas"])
    # GT = get_field(ground_truth, "impressão")
    # PRED = "\n".join(prediction["impressão"])
    GT = get_field(ground_truth, "conduta")
    PRED = "\n".join(prediction["conduta"])

    messages = [
        {
            "role": "system",
            "content": (
                "Você é um especialista em nefrologia com ampla experiência em avaliação de casos clínicos e análises médicas. "
                "Você você irá fornecer avaliações detalhadas e contextualizadas de informações clínicas. "
                # "Seu objetivo é revisar uma consulta de paciente com um médico nefrologista, e analisar duas opiniões médicas distintas sobre a etiologia da doença renal crônica do paciente, "
                "Seu objetivo é revisar o relatório de uma consulta paciente, com base nos dados fornecidos e em duas opiniões médicas distintas sobre como deve ser a conduta ao paciente, "
                # "Seu objetivo é revisar uma consulta de paciente com um médico nefrologista, e analisar duas opiniões médicas distintas acerca da impressão clínica, "
                "e fornecer uma análise sobre a similaridade e precisão dessas duas opiniões. "
                "Use seu conhecimento em nefrologia, diretrizes clínicas e critérios diagnósticos para avaliar a validade e a coerência das informações fornecidas."
            )
        },
        {
            "role": "user",
            "content": (
                "Você receberá um sumário da conversa de uma consulta ambulatorial de um paciente com um médico nefrologia. "
                "Além disso, você receberá as informações completas do paciente, incluindo histórico, exames laboratoriais."
                # "Serão fornecidas duas opiniões médicas distintas sobre a etiologia da doença renal crônica do paciente, "
                "Serão fornecidas duas opiniões médicas distintas sobre a conduta ao paciente"
                # "Serão fornecidas duas opiniões médicas distintas sobre a impressão clínica"
                "e sua tarefa será avaliar o seguinte:\n\n"
                "1. Analisar se as duas opiniões são suficientement semelhantes ou complemtamente diferentes.\n"
                "2. Avaliar se cada uma delas é plausível, dado o contexto clínico do paciente e a conversa da consulta.\n"
                "3. Atribuir um dos seguintes scores com base em sua análise:\n"
                "- Different_Both_Correct: As opiniões são muito diferentes, mas ambas são plausíveis.\n"
                "- Different_first_Correct: As opiniões são muito diferentes, e apenas a primeira é plausível.\n"
                "- Different_second_Correct: As opiniões são muito diferentes, e apenas a segunda é plausível.\n"
                "- Different_Both_Incorrect: As opiniões são muito diferentes, e nenhuma é plausível.\n\n"
                "- Aligned_Correct: As opiniões são suficientemente semelhantes e ambas são plausíveis.\n"
                "- Aligned_Incorrect: As opiniões são suficientemente semelhantes, e nenhuma é plausível.\n"
            )
        },
        {
            "role": "user",
            "content": (
                f"**Sumário da consulta ambulatorial**\n{condensed_dialog}\n\n"
                f"**Informações do Paciente**\n{concatenated_info}\n\n"
                "**Opinião Médica 1**:\n"
                f"{GT}\n\n"
                "**Opinião Médica 2**:\n"
                f"{PRED}\n\n"
                "Com base nas informações acima, avalie e forneça seu score."
                "Justifique sua resposta."
            )
        }
    ]

    # Generate the prompt manually
    prompt = create_chat_prompt(messages)

    outputs = pipeline(
        prompt,
        max_new_tokens=256,
        eos_token_id=pipeline.tokenizer.eos_token_id,
        do_sample=True,
        temperature=1.0,
        top_p=0.9,
    )

    # Print the generated output
    print("GT", GT)
    print("PRED", PRED)
    print(patient_id, outputs[0]["generated_text"][len(prompt):])
    print()